In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import operator
import glob
import os

In [2]:
def get_cpg_positions (refseq, fwd_pos=31, rvs_pos=117):
    pos=list()
    for i,nuc in enumerate(refseq):
        if nuc == 'C':
            if refseq[i+1] == 'G':
                if i>fwd_pos and i<rvs_pos: #exclude primers
                    pos.append(i)
    return(pos)

In [3]:
def get_all_reads (file):
    d = {}
    total_reads = 0
    with open(file) as f:
        for line in f:
            val, seq = line.strip().split("\t")
            d[seq] = int(val)
            
    return(d)

In [4]:
def count_alleles (d, refseq, min_freq=0.001):
    #instantiate new dictionary
    alleles = defaultdict(int)
    
    # count total number of reads
    reads_n = sum(d.values())
    
    # get CpG positions
    pos=get_cpg_positions(refseq)
    
    # amplicon length for 1st filter
    refseq_len=len(refseq)
    
    #minimum number of reads to meet the min freq cutoff for 2nd filter
    min_reads=reads_n*min_freq

    for seq,val in d.items():
        #1st and 2nd filters: exclude all indels, and minimum freq
        if (len(seq) == refseq_len) & (val > min_reads):
            allele=""
            for i,nuc in enumerate(seq):
                if i in pos:
                    allele+=nuc
            testseq=set(allele)
            #3rd filter, for any errors in the positions of interest
            if 'A' not in testseq and 'G' not in testseq: 
                alleles[allele]+=val

    # Sort by number of reads supporting the allele
    alleles_sort = sorted(alleles.items(), key=operator.itemgetter(1), reverse=True)

    # Total number of reads after filtering
    filtered_reads=sum(alleles.values())
    
    return(alleles_sort,filtered_reads)

In [5]:
def group_alleles_by_meCpG (alleles_sort):
    d_group = defaultdict(int)
    for allele,count in alleles_sort:
        d_group[allele.count("C")] +=count
    return(d_group)

In [6]:
def convert_to_df (alleles_sort,filtered_reads,freq_min):
    # get CpG positions
    pos=get_cpg_positions(refseq)
    df=pd.DataFrame()
    df_counts=pd.DataFrame()
    for allele, val in alleles_sort:
        freq=(val/filtered_reads*100)
        if freq >= freq_min: #Frequency cut-offs
            nuc_list = list(allele)
            nuc_list.insert(0,allele)
            row=pd.DataFrame(nuc_list).T
            row.columns=("seq",*pos)
            df= df.append(row,ignore_index=True)
            allele_counts = [allele, freq, val]
            row_counts=pd.DataFrame(allele_counts).T
            row_counts.columns=("seq","freq", "val")
            df_counts = df_counts.append(row_counts,ignore_index=True)

    df_melt = df.melt(id_vars = "seq")
    df_melt_merged = pd.merge(df_melt, df_counts, on="seq")
    
    return(df_melt_merged)

In [7]:
def calculate_meth_fraction (alleles_s, include_unmeth_alleles=True):
    methylated_counts = defaultdict(int)
    methylated_fraction = defaultdict(int)

    totalreads = 0
    for allele,val in alleles_s:
        if include_unmeth_alleles or any('C' == nuc for nuc in allele):
            totalreads += val
            for i,nuc in enumerate(allele):
                if nuc == 'C':
                    methylated_counts[i]+=val
                        
    if not methylated_counts:
        pos=get_cpg_positions(refseq)
        for i,p in enumerate(pos):
             methylated_counts[i]=0
                

    for i,methcount in methylated_counts.items():
        if totalreads == 0:
            methylated_fraction[i] = 0
        else:
            methfrac = methcount/totalreads
            methylated_fraction[i] = methfrac


    df = pd.DataFrame.from_dict(methylated_fraction, orient = "index")
    
    return(df)

In [8]:
def calculate_meth_fraction_min (alleles_sort,filtered_reads,freq_min):
    methylated_counts = defaultdict(int)
    methylated_fraction = defaultdict(int)
    
    pos=get_cpg_positions(refseq)

    val_sum=0
    for allele, val in alleles_sort:
        freq=(val/filtered_reads*100)
        if freq < freq_min: #Frequency cut-offs and second filter, for any errors in the positions of interest
            val_sum+=val
            for i,nuc in enumerate(allele):
                if nuc == 'C':
                    methylated_counts[pos[i]]+=val
                else:
                    methylated_counts[pos[i]]+=0

                    
    for i,methcount in methylated_counts.items():
        methfrac = methcount/val_sum
        methylated_fraction[i] = methfrac

    if not methylated_counts:
        for i,p in enumerate(pos):
             methylated_fraction[i]=0
                
    
    df = pd.DataFrame.from_dict(methylated_fraction, orient = "index")
    df.reset_index(level=0, inplace=True)
    df.columns = ['variable', 'shade']
    df['value'] = 'below_min'
    df['seq'] = 'below_min'
    df['freq'] = (val_sum/filtered_reads*100)
    df['val'] = val_sum

        
    return(df)

In [9]:
def plot_lollipop_colour (df, outpath, outname="All_samples_combined_colour.pdf"):  
    # Changing default font to Arial
    plt.rcParams['font.sans-serif'] = "Arial"
    plt.rcParams['font.family'] = "sans-serif"
    
    df_melt = df.melt(id_vars="pos")
    df_melt['variable']= df_melt["variable"].str.split('_').str[0]
    df_melt = df_melt.sort_index(ascending=False)
    #df_melt = df_melt.sort_values(by=['variable'])

    plt.set_cmap('coolwarm')
    plt.figure()
    fig, ax = plt.subplots(figsize=(5,4))
    ax.hlines(df_melt['variable'],min(df_melt['pos']),max(df_melt['pos']), label='_nolegend_', zorder=1)
    im = ax.scatter(df_melt['pos'],df_melt['variable'],label="meC", c=df_melt['value'],edgecolors ="black", s=50, zorder=2)
    im.set_clim(0,1)
    cbar = fig.colorbar(im, ax=ax,ticks=[0.1, 0.5, 0.9])
    cbar.ax.tick_params(labelsize=6.5, # label size
                        length=1.5, # length of ticks
                        pad = 0.4) # distance between ticks and labels

    ax.axes.set_xticks(list(df_melt['pos'].unique()))
    ax.tick_params(axis='x', which='major', labelsize=6.5, rotation=45)
    ax.tick_params(axis='y', which='major', labelsize=6.5)
    
    plt.tight_layout()
    fig.savefig(outpath + outname)
    plt.close()

In [10]:
def plot_lollipop (df,sname,outpath,freq_min):
    
    # Changing default font to Arial
    plt.rcParams['font.sans-serif'] = "Arial"
    plt.rcParams['font.family'] = "sans-serif"
    
    plt.set_cmap('coolwarm')

    df_C = df[df['value']=="C"]
    df_T = df[df['value']=="T"]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10,5))

    ax1.hlines(df['seq'],min(df['variable']),max(df['variable']), label='_nolegend_', zorder=1)
    ax1.scatter(df_C['variable'],df_C['seq'],label="meC", color="#B30326",edgecolors ="black", s=50, zorder=2)
    ax1.scatter(df_T['variable'],df_T['seq'],label="C", color="#3A4CC0",edgecolors ="black", s=50, zorder=3)
    ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2)
    ax1.axes.set_xticks(list(df['variable'].unique()))
    ax1.tick_params(axis='x', which='major', labelsize=6.5, rotation=45)
    ax1.axes.set_xlabel("RAD51C CpG site")

    ax2.barh(df['seq'], df['freq'], align='center', color='grey')
    ax2.axes.set_yticks([])
    ax2.axes.set_xlabel("Frequency (%)")
    ax2.set_xlim([0, 100])

    plt.suptitle(sname)# + f"\nMethylation alleles detected at >{freq_min}% frequency") 

    fig.tight_layout(rect=[0, 0.03, 1, 0.9])

    fig.savefig(f"{outpath}{sname}_{freq_min}perc_barplot.pdf")
    
    plt.close()

In [11]:
def plot_lollipop_combined (df,df_below_freq,sname,outpath,freq_min, colbar=True):

    # Changing default font to Arial
    plt.rcParams['font.sans-serif'] = "Arial"
    plt.rcParams['font.family'] = "sans-serif"

    plt.set_cmap('coolwarm')

    df_C = df[df['value']=="C"]
    df_T = df[df['value']=="T"]

    hb=df['seq'].drop_duplicates().count()

    fig, ((ax3, ax4),(ax1, ax2)) = plt.subplots(2, 2, sharex='col',sharey='row',
                                                gridspec_kw=dict(width_ratios=[10,5], height_ratios=[1,hb],hspace=0))    

    # Merged epialleles
    ax3.hlines(df_below_freq['seq'],min(df_below_freq['variable']),max(df_below_freq['variable']), label='_nolegend_', zorder=1)
    im = ax3.scatter(df_below_freq['variable'],df_below_freq['seq'],label='_nolegend_', c=df_below_freq['shade'],edgecolors ="black", s=50, zorder=2)
    im.set_clim(0,1)
    ax3.tick_params(axis='x', which='major', labelsize=8, rotation=45)
    ax3.axes.set_xticks([])

    # Merged epialleles - frequency
    ax4.barh(df_below_freq['seq'], df_below_freq['freq'], align='center', color='grey')
    ax4.axes.set_yticks([])
    ax4.set_xlim(0,100)

    # Individual epialleles
    ax1.hlines(df['seq'],min(df['variable']),max(df['variable']), label='_nolegend_', zorder=1)
    ax1.scatter(df_C['variable'],df_C['seq'],label="meC", color="#B30326",edgecolors ="black", s=50, zorder=2)
    ax1.scatter(df_T['variable'],df_T['seq'],label="C", color="#3A4CC0",edgecolors ="black", s=50, zorder=3)
    ax1.axes.set_xticks(list(df['variable'].unique()))
    ax1.tick_params(axis='x', which='major', labelsize=6.5, rotation=45)
    ax1.axes.set_xlabel("RAD51C CpG site",size=8,labelpad=4)
    ax1.axes.set_yticks([])

    # Individual epialleles - frequency
    ax2.barh(df['seq'], df['freq'], align='center', color='grey')
    ax2.axes.set_yticks([])
    ax2.tick_params(axis='x', labelsize=6.5)
    ax2.axes.set_xlabel("Frequency (%)",size=8,labelpad=10)
    ax2.set_xlim(0,100)

    #Text labels
    ax3.text(-0.135,0.4, f"<{freq_min}% frequency \n(merged)", size=6.5, ha="center", 
            transform=ax3.transAxes)
    ax1.text(-0.14,0.9, f"≥{freq_min}% frequency", size=6.5, ha="center", 
            transform=ax1.transAxes)

    # Figure title
    fig.suptitle(sname, size=8, weight='bold')

    fig.tight_layout(rect=[0, 0.03, 1, 0.9])
    
    if colbar:
        # After tight layout, otherwise issues warnings
        # Controlling the placement of the colour bar
        adj_par = len(df.seq.unique())
        adj_height=(2.5+adj_par*2.5)
        adj_bbox_coord=(1.0436+0.0432*adj_par)

        axins = inset_axes(ax3,
                          width="60%",  
                         height=f"{adj_height}%",
                          #height="7.5%",  
                          loc='lower left',
                          bbox_to_anchor=(0, adj_bbox_coord, 1, 1), 
                          #bbox_to_anchor=(0, 1.25963685, 1, 1), 
                          bbox_transform=ax3.transAxes,
                          borderpad=0)
        # Colour bar
        cbar = fig.colorbar(im, 
                            #label='CpG methylation',
                            ax=ax3,
                            cax=axins, 
                            ticks=[0.1, 0.5, 0.9],
                            orientation="horizontal")

        cbar.ax.tick_params(labelsize=6.5, # label size
                            length=1.5, # length of ticks
                            pad = 0.4) # distance between ticks and labels

        cbar.set_label(label='CpG methylation',
                       labelpad=-25,
                       size=8)
        
        fig.savefig(f"{outpath}{sname}_{freq_min}perc_barplot.pdf")
    else:
        fig.savefig(f"{outpath}{sname}_{freq_min}perc_barplot_nolegend.pdf")
        

    plt.close()

In [16]:
# !define reference sequence
refseq="GAAAATTTACAAGACTGCGCAAAGCTGCAAGGCCCGGAGCCCCGTGCGGCCAGGCCGCAGA\
GCCGGCCCCTTCCGCTTTACGTCTGACGTCACGCCGCACGCCCCAGCGAGGGCGTGCGGAGTTTGGCTGCTCCGGGGTTAG" 
# !define paths
path="/Users/olgak/Documents/Postdoc_Waddell/Misc/2020_WFH/Scott_-_Ovarian_cancer/data/"
file_path=path + "processed/amplicon_meth/"
outpath=file_path + "notebook_filt_no_cutoff/"

if not os.path.exists(outpath):
    os.mkdir(outpath)
    
    
# !define sample ids to sample labels
labels_df = pd.read_csv(path + "supporting/SampleID_labels_amplicon_meth.csv", index_col = 0)

In [17]:
df_group_meCpG_comb = []

for i,file in enumerate(glob.glob(file_path + "*_grp")):
    # Extract sample label
    sid=os.path.basename(file).replace("_L001_merged_seqprep.fastq.fna_1_num_grp","")
    sname=labels_df.loc[sid]['ShortLabel']
    if pd.isnull(sname):
        sname=labels_df.loc[sid]['SampleLabel']
    
    # Generate dictionary with all reads
    d=get_all_reads(file=file)

    
    # Count only CpG sites in alleles
    alleles_sort,filtered_reads=count_alleles(d,refseq, min_freq=0)
        
    df_alleles_sort = pd.DataFrame(alleles_sort, columns=['allele',sname])

    df_alleles_sort = df_alleles_sort.set_index('allele')
    if i == 0:
        df_alleles_sort_all=df_alleles_sort
    elif i > 0:
        df_alleles_sort_all=df_alleles_sort_all.join(df_alleles_sort, how="outer")

    #Calculate methylation fraction per CpG site
    df_sample=calculate_meth_fraction(alleles_sort)
    df_sample_unmeth=calculate_meth_fraction(alleles_sort, include_unmeth_alleles=False)
    df_sample.columns=[sname]
    df_sample_unmeth.columns=[sname]
    if i == 0:
        df_alt=df_sample
        df_alt_unmeth=df_sample_unmeth
    elif i > 0:
        df_alt=df_alt.join(df_sample)
        df_alt_unmeth=df_alt_unmeth.join(df_sample_unmeth)

    # Prepare individual sample plots grouping alleles <2% and 5%
    for freq_min in [5]:
        df=convert_to_df(alleles_sort,filtered_reads,freq_min)
        df_below_freq=calculate_meth_fraction_min(alleles_sort,filtered_reads,freq_min)
        df.variable = df.variable - 156
        df_below_freq.variable = df_below_freq.variable - 156
        plot_path=f'{outpath}{freq_min}_perc/'
        if not os.path.exists(plot_path):
            os.mkdir(plot_path)
        if df_below_freq.freq.sum() > 0:           
            plot_lollipop_combined(df,df_below_freq,sname,plot_path,freq_min)
            plot_path2=f'{outpath}{freq_min}_perc_no_legend/'
            if not os.path.exists(plot_path2):
                os.mkdir(plot_path2) 
            plot_lollipop_combined(df,df_below_freq,sname,plot_path2,freq_min, colbar=False)
        else:
            plot_lollipop(df,sname,plot_path,freq_min)
            
        




<Figure size 640x480 with 0 Axes>

In [18]:
df_alleles_sort_all.to_csv(outpath + "df_alleles_sort_all.csv")
df_alt.to_csv(outpath + "df_meth_freq.csv")
df_alt_unmeth.to_csv(outpath + "df_exclude_unmeth-alleles_freq.csv")

In [19]:
df_alt = df_alt.filter(labels_df.dropna().ShortLabel.tolist())

df_alt_unmeth = df_alt_unmeth.filter(labels_df.dropna().ShortLabel.tolist())

pos_promoter = list(map(lambda x: x - 156, get_cpg_positions(refseq)))
df_alt['pos'] = pos_promoter
df_alt_unmeth['pos'] = pos_promoter


plt.style.use('default')
plot_lollipop_colour(df=df_alt, outpath=outpath,
                    outname="All_samples_combined_colour.pdf")

plot_lollipop_colour(df=df_alt_unmeth, outpath=outpath,
                     outname="All_samples_combined_exclude_unmeth-alleles_colour.pdf")

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>